In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# data import 
data = pd.read_csv("", sep = ";")
order = data['Unnamed: 1']
data = data.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1)
data = data.replace('  ', '0').fillna('0').astype('string')
data = data.replace(0, '0')
pd.set_option('display.max_rows', 10000)

In [ ]:
# first_player = []
# for i in range(len(order)):
#     first_player.append(int(order[i][0]))

In [ ]:
df = data.drop(data.columns[[0, 3, 6, 9, 12, 15, 23]], axis = 1).reset_index(drop=True)

In [ ]:
df = df[(df.P1H2 != '0') & (df.P2H2 != '0') & (df.P3H2 != '0') & (df.P4H2 != '0') & (df.P5H2 != '0') & (df.P6H2 != '0')].reset_index(drop=True)

In [ ]:
def count_match(arr1, arr2):
    num_matches = 0
    for i in range(len(arr1)):
        if arr2[i] in arr1 and arr2[i] != '0':
            num_matches += 1
            arr1.remove(arr2[i])
    return num_matches

In [ ]:
matches = [0]
for i in range(len(df) - 1):
    matches.append(count_match(list(df.loc[i]), list(df.loc[i + 1])))

In [ ]:
df['matches'] = matches
df

In [ ]:
def encode_index(i):
    if i == 1:
        return 1
    elif i == 3:
        return 2
    elif i == 5:
        return 3
    elif i == 7:
        return 4
    elif i == 9:
        return 5
    elif i == 11:
        return 6

In [ ]:
left_players = []
real_orders = []
for i in range(len(df)):
    non_zero = []
    for j in range(1, 12, 2):
        if df.iloc[i, j] != '0':
            non_zero.append(encode_index(j))
    left_players.append(non_zero)
    # cut_point = non_zero.index(first_player[i])
    # real_order = non_zero[cut_point:] + non_zero[:cut_point]
    # real_orders.append(real_order)

In [ ]:
df_card_pos = pd.DataFrame(columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [ ]:
class players_hand:
    def __init__(self, arr_table):
        self.p1 = (arr_table[0], arr_table[1])
        self.p2 = (arr_table[2], arr_table[3])
        self.p3 = (arr_table[4], arr_table[5])
        self.p4 = (arr_table[6], arr_table[7])
        self.p5 = (arr_table[8], arr_table[9])
        self.p6 = (arr_table[10], arr_table[11])
        self.flop = (arr_table[12], arr_table[13], arr_table[14])
        self.turn = arr_table[15]
        self.river = arr_table[16]

    # def sort_pl(self, order):
    #     arr = []
    #     for i in range(len(order)):
    #         if order[i] == 1:
    #             arr.append(self.p1)
    #         if order[i] == 2:
    #             arr.append(self.p2)
    #         if order[i] == 3:
    #             arr.append(self.p3)
    #         if order[i] == 4:
    #             arr.append(self.p4)
    #         if order[i] == 5:
    #             arr.append(self.p5)
    #         if order[i] == 6:
    #             arr.append(self.p6)
    #     for i in range(6 - len(arr)):
    #         arr.append('0')
    #     return arr


In [ ]:
arr_encoded = []
for i in range(len(df)):
    arr_encoded.append(players_hand(df.loc[i]))

In [ ]:
df

In [ ]:
def right_pos_df(arr_encoded):
    arr_df = []
    for i in range(len(arr_encoded)):
        table = arr_encoded[i]
        arr = []
        for j in range(len(left_players[i])):
            if left_players[i][j] == 1:
                arr.append(table.p1[0])
                arr.append(table.p1[1])
            if left_players[i][j] == 2:
                arr.append(table.p2[0])
                arr.append(table.p2[1])
            if left_players[i][j] == 3:
                arr.append(table.p3[0])
                arr.append(table.p3[1])
            if left_players[i][j] == 4:
                arr.append(table.p4[0])
                arr.append(table.p4[1])
            if left_players[i][j] == 5:
                arr.append(table.p5[0])
                arr.append(table.p5[1])
            if left_players[i][j] == 6:
                arr.append(table.p6[0])
                arr.append(table.p6[1])
        for m in range(12 - len(arr)):
            arr.append(0)

        arr.append(arr_encoded[i].flop[0])
        arr.append(arr_encoded[i].flop[1])
        arr.append(arr_encoded[i].flop[2])
        arr.append(arr_encoded[i].turn)
        arr.append(arr_encoded[i].river)
    
        arr_df.append(arr)
    df_pl_pos = pd.DataFrame(columns=['P1H1', 'P1H2', 'P2H1', 'P2H2', 'P3H1', 'P3H2', 'P4H1', 'P4H2', 'P5H1', 'P5H2', 'P6H1', 'P6H2', 'Flop1', 'Flop2', 'Flop3', 'Turn', 'River'], data = arr_df)
    return df_pl_pos

In [ ]:
d = right_pos_df(arr_encoded)

In [ ]:
n = 2538 * 2
df_pl = df.apply(lambda x: x.value_counts()).fillna(0)
df_freq = pd.DataFrame(columns=[1, 2, 3, 4, 5, 6], index=['AS', 'AD', 'AC', 'AH', 'KS', 'KD', 'KC', 'KH', 'QS', 'QD', 'QC', 'QH', 'JS', 'JD', 'JC', 'JH', 'TS', 'TD', 'TC', 'TH', 
                                                    '9S', '9D', '9C', '9H', '8S', '8D', '8C', '8H', '7S', '7D', '7C', '7H', '6S', '6D', '6C', '6H', '5S', '5C', '5D', '5H',
                                                    '4S', '4D', '4C', '4H', '3S', '3D', '3C', '3H', '2S', '2D', '2C', '2H'])
# df_freq[1] = (df_pl['P1H1'] + df_pl['P1H2']) / n
# df_freq[2] = (df_pl['P2H1'] + df_pl['P2H2']) / n
# df_freq[3] = (df_pl['P3H1'] + df_pl['P3H2']) / (n - 750)
# df_freq[4] = (df_pl['P4H1'] + df_pl['P4H2']) / (n - 1850)
# df_freq[5] = (df_pl['P5H1'] + df_pl['P5H2']) / (n - 2840)
# df_freq[6] = (df_pl['P6H1'] + df_pl['P6H2']) / (n - 3700)

df_freq[1] = (df_pl['P1H1'] + df_pl['P1H2'])
df_freq[2] = (df_pl['P2H1'] + df_pl['P2H2'])
df_freq[3] = (df_pl['P3H1'] + df_pl['P3H2'])
df_freq[4] = (df_pl['P4H1'] + df_pl['P4H2'])
df_freq[5] = (df_pl['P5H1'] + df_pl['P5H2'])
df_freq[6] = (df_pl['P6H1'] + df_pl['P6H2'])

In [ ]:
def fig_distr(i):
    thres = df_freq[i].sum() / 52
    fig = plt.figure(figsize=(12, 8))
    graph = sns.barplot(x = df_freq[i].index, y = df_freq[i], width = 0.9)
    graph.axhline(thres, color='black', dashes=(5, 2), label = 'threshold')
    graph.set(ylim=(0, 40))
    plt.ylabel('Frequency')
    plt.title('Player ' + str(i))
    plt.legend()
    plt.tight_layout()

In [ ]:
fig_distr(2)

In [ ]:
import scipy.stats as stats
chi_stats, pvals = stats.chisquare(df_freq)

In [ ]:
chi_stats

In [ ]:
def cards_distribution_graph(df_card_stats):    
    fig = plt.figure(figsize=(25, 30))
    ax1 = fig.add_subplot(11, 1, 1)
    ax1.bar(df_card_stats.index, df_card_stats.iloc[:, 0], color = deck_values_colors)
    ax1.set_title(df_card_stats.columns[0])
    for i in range(1, df_card_stats.shape[1]):
        ax = fig.add_subplot(11, 1, i + 1, sharey = ax1)
        ax.bar(df_card_stats.index, df_card_stats.iloc[:, i], color = deck_values_colors)
        ax.set_title(df_card_stats.columns[i])
    plt.tight_layout()

In [ ]:
arr_df = []
for i in range(len(real_orders)):
    # arr = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    arr = []
    for j in range(len(real_orders[i])):
        if real_orders[i][j] == 1:
            arr.append(arr_encoded[i].p1[0])
        if real_orders[i][j] == 2:
            arr.append(arr_encoded[i].p2[0])
        if real_orders[i][j] == 3:
            arr.append(arr_encoded[i].p3[0])
        if real_orders[i][j] == 4:
            arr.append(arr_encoded[i].p4[0])
        if real_orders[i][j] == 5:
            arr.append(arr_encoded[i].p5[0])
        if real_orders[i][j] == 6:
            arr.append(arr_encoded[i].p6[0])

    for j in range(len(real_orders[i])):
        if real_orders[i][j] == 1:
            arr.append(arr_encoded[i].p1[1])
        if real_orders[i][j] == 2:
            arr.append(arr_encoded[i].p2[1])
        if real_orders[i][j] == 3:
            arr.append(arr_encoded[i].p3[1])
        if real_orders[i][j] == 4:
            arr.append(arr_encoded[i].p4[1])
        if real_orders[i][j] == 5:
            arr.append(arr_encoded[i].p5[1])
        if real_orders[i][j] == 6:
            arr.append(arr_encoded[i].p6[1])
    # zeroes = list(np.full(12 - len(arr), '0'))
    # arr += zeroes
    arr.append('|')
    arr.append(arr_encoded[i].flop[0])
    arr.append(arr_encoded[i].flop[1])
    arr.append(arr_encoded[i].flop[2])
    arr.append('0')
    arr.append(arr_encoded[i].turn)
    arr.append('0')
    arr.append(arr_encoded[i].river)
    arr = np.pad(arr, (0, 20-len(arr)), 'constant')
    arr_df.append(arr)
    

In [ ]:
df_card_pos = pd.DataFrame(columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], data = arr_df)

In [ ]:
df_freq = df_card_pos.apply(lambda x: x.value_counts()).reindex(index=['AH', 'AC', 'AD', 'AS', 'KH', 'KC', 'KD', 'KS', 'QH', 'QC', 'QD', 'QS', 'JH', 'JC', 'JD', 'JS', 'TH', 'TC', 'TD', 'TS', 
                                                    '9H', '9C', '9D', '9S', '8H', '8C', '8D', '8S', '7H', '7C', '7D', '7S', '6H', '6C', '6D', '6S', '5H', '5C', '5D', '5S',
                                                    '4H', '4C', '4D', '4S', '3H', '3C', '3D', '3S', '2H', '2C', '2D', '2S'])